In [1]:
import os
import numpy as np
import rasterio
from matplotlib.colors import LinearSegmentedColormap


# ----------------------------------------------------------
# 1. DJI NDVI Colormap definieren
# ----------------------------------------------------------

def get_dji_colormap():
    """
    Reproduziert den DJI NDVI-Look als kontinuierliche Colormap.
    Skala: -1 .. +1
    """

    # NDVI-Wert + Farbe
    points = [
        (-1.0,  "#000000"),  # schwarz
        (-0.5,  "#444444"),  # dunkelgrau
        (-0.05, "#808080"),  # mittelgrau
        ( 0.00, "#d0d0d0"),  # hellgrau
        ( 0.02, "#8F00FF"),  # violett band
        ( 0.25, "#00FF00"),  # grün
        ( 0.50, "#F8FF00"),  # gelb-grün
        ( 0.75, "#FF4D00"),  # orange-rot
        ( 0.85, "#FF0000"),  # rot
        ( 1.00, "#FF00FF"),  # pink/magenta
    ]

    # auf 0-1 normalisieren
    xs = [(v + 1.0) / 2.0 for v, _ in points]
    cols = [c for _, c in points]

    return LinearSegmentedColormap.from_list("dji_ndvi", list(zip(xs, cols)), N=256)


dji_cmap = get_dji_colormap()


# ----------------------------------------------------------
# 2. Funktion: NDVI → DJI-RGB-GeoTIFF
# ----------------------------------------------------------

def convert_ndvi_to_dji_rgb(in_path, out_path):
    with rasterio.open(in_path) as src:
        arr = src.read(1).astype("float32")
        profile = src.profile.copy()
        nodata = src.nodata

    # NDVI erkennen: falls 0..65535 drohnenindex, konvertieren
    if arr.max() > 1.5:
        ndvi = (arr / 32768.0) - 1.0
    else:
        ndvi = arr

    # NoData → NaN
    if nodata is not None:
        ndvi = np.where(arr == nodata, np.nan, ndvi)

    # Wertebereich [-1..1] erzwingen
    ndvi = np.clip(ndvi, -1.0, 1.0)

    # NaN ersetzen → schwarz
    ndvi_fill = np.where(np.isnan(ndvi), -1.0, ndvi)

    # normalisieren für colormap
    norm = (ndvi_fill + 1.0) / 2.0
    norm = np.clip(norm, 0.0, 1.0)

    rgba = dji_cmap(norm)
    rgb = (rgba[:, :, :3] * 255).astype("uint8")

    # GeoTIFF Profil anpassen
    profile.update(
        dtype="uint8",
        count=3,
        nodata=0,
        photometric="RGB"
    )

    with rasterio.open(out_path, "w", **profile) as dst:
        for i in range(3):
            dst.write(rgb[:, :, i], i + 1)


# ----------------------------------------------------------
# 3. Batch-Konvertierung aller TIFFs im Ordner
# ----------------------------------------------------------

input_folder  = "/home/gast/Khanh/Repositories/yolov12/test"
output_folder = "/home/gast/Khanh/Repositories/yolov12/test_dji_rgb"

os.makedirs(output_folder, exist_ok=True)

for fname in sorted(os.listdir(input_folder)):
    if not fname.lower().endswith((".tif", ".tiff")):
        continue

    in_f  = os.path.join(input_folder, fname)
    out_f = os.path.join(output_folder, fname.replace(".tif", "_DJI_RGB.tif"))

    print(f"Konvertiere: {fname}")
    convert_ndvi_to_dji_rgb(in_f, out_f)

print("Fertig.")


Konvertiere: ausgabe_ndvi_16.tif
Konvertiere: ausgabe_ndvi_20.tif
Konvertiere: ausgabe_ndvi_26.tif
Konvertiere: ausgabe_ndvi_4.tif
Konvertiere: ausgabe_ndvi_46.tif
Konvertiere: ausgabe_ndvi_48.tif
Fertig.
